<a href="https://colab.research.google.com/github/FKz11/NLP/blob/main/lesson_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson-5

Тема «POS-tagger и NER»

Задание 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы, сделать выводы
 
Задание 2. Проверить, насколько хорошо работает NER
Данные брать из http://www.labinform.ru/pub/named_entities/
проверить NER из nltk/spacy/deeppavlov.
написать свой NER, попробовать разные подходы.
передаём в сетку токен и его соседей.
передаём в сетку только токен.
свой вариант.
сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


### Библиотеки:

In [1]:
!pip install -qq pyconll

In [2]:
import pyconll

from collections import Counter

import numpy as np

from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras import activations
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalAveragePooling1D, BatchNormalization, Dropout, LSTM, Activation

In [3]:
import warnings
warnings.filterwarnings("ignore")

## Задание 1

### Данные:

In [4]:
!mkdir dataset_ru

данные взяты из 

https://github.com/UniversalDependencies/UD_Russian-SynTagRus

In [5]:
!wget -O ./dataset_ru/ru_syntagrus-ud-train-a.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./dataset_ru/ru_syntagrus-ud-train-b.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
!wget -O ./dataset_ru/ru_syntagrus-ud-train-c.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-c.conllu
!wget -O ./dataset_ru/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2022-09-05 15:55:12--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736565 (39M) [text/plain]
Saving to: ‘./dataset_ru/ru_syntagrus-ud-train-a.conllu’

./dataset_ru/ru_syn 100%[===================>]  38.85M  --.-KB/s    in 0.09s   

2022-09-05 15:55:16 (411 MB/s) - ‘./dataset_ru/ru_syntagrus-ud-train-a.conllu’ saved [40736565/40736565]

--2022-09-05 15:55:16--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

Тэги:

1) ADJ - adjective (прилагательное)

The 10 most frequent ADJ lemmas: новый, другой, первый, самый, должен, российский, большой, сам, нужный, последний

2) ADP - adposition

The 10 most frequent ADP lemmas: в, на, с, по, к, из, о, для, за, от

3) ADV - adverb (наречие)

The 10 most frequent ADV lemmas: так, уже, еще, можно, как, более, очень, где, однако, там

4) AUX - auxiliary (вспомогательный)

The 10 most frequent AUX lemmas: быть, бы

5) CCONJ - coordinating conjunction (координирующее соединение)

The 10 most frequent CCONJ lemmas: и, а, но, или, ни, да, либо, причем, зато, только

6) DET - determiner (определитель)

The 10 most frequent DET lemmas: этот, свой, весь, тот, такой, его, наш, их, мой, какой

7) INTJ - interjection (междометие)

The 10 most frequent INTJ lemmas: о, ах, а, гм, ой, ох, эх, ура, хахаха, ага

8) NOUN - noun (существительное)

The 10 most frequent NOUN lemmas: год, человек, время, страна, дело, жизнь, работа, власть, система, день

9) NUM - numeral (числительное)

The 10 most frequent NUM lemmas: один, два, много, несколько, три, 1, 10, 20, четыре, 2

10) PART - particle (частица)

The 10 most frequent PART lemmas: не, и, же, только, даже, вот, ли, именно, лишь, просто

11) PRON - pronoun (местоимение)

The 10 most frequent PRON lemmas: он, это, я, который, они, то, мы, она, что, все

12) PROPN - proper noun (имя собственное)

The 10 most frequent PROPN lemmas: Россия, Москва, США, Путин, СССР, Европа, Владимир, Германия, Александр, Сергей

13) PUNCT - punctuation (пунктуация)

The 10 most frequent PUNCT lemmas: ,, ., “, -, :, ?, ), (, !, …

14) SCONJ - subordinating conjunction (подчиняющий союз)

The 10 most frequent SCONJ lemmas: что, как, если, чтобы, когда, то, чем, хотя, поскольку, пока

15) SYM - symbol (символ)

The 10 most frequent SYM lemmas: %, $, №, °, &, €, +, =, №№, x

16) VERB - verb (глагол)

The 10 most frequent VERB lemmas: мочь, быть, стать, говорить, сказать, делать, иметь, знать, идти, нет

17) X - other (другое)

The 10 most frequent X lemmas: of, the, and, artist, empire, in, V&A, X, for, i

In [6]:
full_train = pyconll.load_from_file('dataset_ru/ru_syntagrus-ud-train-a.conllu')
full_train_b = pyconll.load_from_file('dataset_ru/ru_syntagrus-ud-train-b.conllu')
full_train_c = pyconll.load_from_file('dataset_ru/ru_syntagrus-ud-train-c.conllu')

full_train.extend([*full_train_b, *full_train_c]) # объединяем

full_test = pyconll.load_from_file('dataset_ru/ru_syntagrus-ud-dev.conllu')

In [7]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [8]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [9]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len("" if token.form is None else token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [10]:
all_train_texts = [' '.join("" if token.form is None else token.form for token in sent) for sent in full_train]
all_test_texts = [' '.join("" if token.form is None else token.form for token in sent) for sent in full_test]

all_train_labels = [' '.join("" if token.upos is None else token.upos for token in sent) for sent in full_train]
all_test_labels = [' '.join("" if token.upos is None else token.upos for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[:10]))
print('\n'.join(all_train_labels[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [11]:
example_tag = fdata_sent_test[0]
example_tag

['Алгоритм',
 ',',
 'от',
 'имени',
 'учёного',
 'аль',
 '-',
 'Хорезми',
 ',',
 '-',
 'точный',
 'набор',
 'инструкций',
 ',',
 'описывающих',
 'порядок',
 'действий',
 'исполнителя',
 'для',
 'достижения',
 'результата',
 'решения',
 'задачи',
 'за',
 'конечное',
 'время',
 '.']

### UnigramTagger

In [12]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(example_tag), unigram_tagger.evaluate(fdata_test))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', 'ADJ'),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.8782863467673677

### BigramTagger

In [13]:
bigram_tagger = BigramTagger(fdata_train)
display(bigram_tagger.tag(example_tag), bigram_tagger.evaluate(fdata_test))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', None),
 ('аль', None),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', None),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', None),
 ('.', 'PUNCT')]

0.7101308678950452

Результат хуже, так как многие биграмы довольно редкие и не встречались в тренировочных данных, поэтому для таких биграм нужно использовать униграмы в качестве backoff:

In [14]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.tag(example_tag), bigram_tagger.evaluate(fdata_test))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', 'ADJ'),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.8839768214076438

### TrigramTagger

In [15]:
trigram_tagger = TrigramTagger(fdata_train)
display(trigram_tagger.tag(example_tag), trigram_tagger.evaluate(fdata_test))

[('Алгоритм', None),
 (',', None),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', None),
 ('аль', None),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', None),
 ('-', 'PUNCT'),
 ('точный', None),
 ('набор', None),
 ('инструкций', None),
 (',', 'PUNCT'),
 ('описывающих', None),
 ('порядок', None),
 ('действий', None),
 ('исполнителя', None),
 ('для', 'ADP'),
 ('достижения', None),
 ('результата', None),
 ('решения', None),
 ('задачи', None),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', None),
 ('.', 'PUNCT')]

0.4067191874470994

Тут такая же ситуация, как и с биграмами, теперь воспользуемся в качестве backoff биграмами, которые в свою очередь имеют backoff униграмы:

In [16]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger.tag(example_tag), trigram_tagger.evaluate(fdata_test))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', 'ADJ'),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.8830522820496126

Результаты чуть хуже, чем у биграм с backoff униграм

### Комбинация тэггеров

In [17]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NOUN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
display(tag.tag(example_tag), tag.evaluate(fdata_test))

[('Алгоритм', 'NOUN'),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', 'NOUN'),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', 'ADJ'),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.9119799466111075

Результат получился лучше всех, так как оказывается, что даже некоторых слов не было в тренировочных данных, поэтому при добавлении DefaultTagger('NOUN'), который неизвестным словам ставит в соответсвие сущиствительное, так как это предположительно самая частая часть речи. 

### Свой тэггер

Напишем нейронную сеть

Предобработка данных:

In [18]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    norm_sent = []
    label_sent = []
    for tok in sent:
        if (tok[0] is None) or (tok[1] is None):
            continue
        norm_sent.append(tok[0])
        label_sent.append(tok[1])
    train_tok.append(norm_sent)
    train_label.append(label_sent)
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    norm_sent = []
    label_sent = []
    for tok in sent:
        if (tok[0] is None) or (tok[1] is None):
            continue
        norm_sent.append(tok[0])
        label_sent.append(tok[1])
    test_tok.append(norm_sent)
    test_label.append(label_sent)

In [19]:
corpus = [token for sent in train_tok for token in sent]
corpus[:5]

['Анкета', '.', 'Начальник', 'областного', 'управления']

In [20]:
freq_dict = Counter(corpus)
freq_dict_sorted = sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[(',', 97978),
 ('.', 64114),
 ('в', 30986),
 ('и', 30064),
 ('"', 21639),
 ('-', 19280),
 ('не', 15028),
 ('на', 14637),
 ('что', 10590),
 ('с', 9559)]

In [21]:
vocabulary_size = len(freq_dict)
vocabulary_size

132847

In [22]:
VOCABULARY_SIZE = vocabulary_size
VOCABULARY_SIZE

132847

In [23]:
top_n_words = [word for word, freq in freq_dict_sorted[:VOCABULARY_SIZE]]
top_n_words[:5]

[',', '.', 'в', 'и', '"']

In [24]:
vocab = {y:x for x,y in enumerate(top_n_words, start=1)}
list(vocab.items())[:5]

[(',', 1), ('.', 2), ('в', 3), ('и', 4), ('"', 5)]

In [25]:
train_tok_num = [[vocab.get(i, 0) for i in j] for j in train_tok]
test_tok_num = [[vocab.get(i, 0) for i in j] for j in test_tok]
print(train_tok[0])
print(train_tok_num[0])

['Анкета', '.']
[39888, 2]


In [26]:
max_len = max(len(sent) for sent in train_tok_num)
max_len

205

In [27]:
MAX_LEN = max_len
MAX_LEN

205

In [28]:
x_train_nn = [i+[0]*(MAX_LEN-len(i)) for i in train_tok_num]
x_test_nn = [i+[0]*(MAX_LEN-len(i)) for i in test_tok_num]
x_train_nn[0][:10]

[39888, 2, 0, 0, 0, 0, 0, 0, 0, 0]

In [29]:
x_train_nn_array = np.array([[j for j in i] for i in x_train_nn])
x_test_nn_array = np.array([[j for j in i] for i in x_test_nn])
x_train_nn_array[0]

array([39888,     2,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [30]:
tags = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']

In [31]:
num2tag = dict(zip(range(1, len(tags)+1), tags))
num2tag

{1: 'ADJ',
 2: 'ADP',
 3: 'ADV',
 4: 'AUX',
 5: 'CCONJ',
 6: 'DET',
 7: 'INTJ',
 8: 'NOUN',
 9: 'NUM',
 10: 'PART',
 11: 'PRON',
 12: 'PROPN',
 13: 'PUNCT',
 14: 'SCONJ',
 15: 'SYM',
 16: 'VERB',
 17: 'X'}

In [32]:
tag2num = {j:i for i,j in num2tag.items()}
tag2num

{'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'PUNCT': 13,
 'SCONJ': 14,
 'SYM': 15,
 'VERB': 16,
 'X': 17}

In [33]:
y_train_nn = [[tag2num[i] for i in j]+[0]*(MAX_LEN-len(j)) for j in train_label]
y_test_nn = [[tag2num[i] for i in j]+[0]*(MAX_LEN-len(j)) for j in test_label]
y_train_nn[0][:10]

[8, 13, 0, 0, 0, 0, 0, 0, 0, 0]

In [34]:
y_train_nn_array = np.array([[j for j in i] for i in y_train_nn])
y_test_nn_array = np.array([[j for j in i] for i in y_test_nn])
y_train_nn_array[0]

array([ 8, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0])

Модель:

In [35]:
embedding_dim = 64
hidden_dim = 32
dropout = 0.2

Загрузим предложения целиком и будем предсказывать сразу много меток, а не одну, как если бы мы передовали по одному слову.

Включим lstm слой, с помощью его, модель должна улавливать контекст и преобретать логику, а не просто смотреть на частоту встречания части речи и данного слова


In [36]:
model = Sequential([
    Embedding(VOCABULARY_SIZE+1, embedding_dim), # VOCABULARY_SIZE+1 так как еще 0, который показывает где нет слова
    BatchNormalization(),
    Dropout(dropout),

    LSTM(embedding_dim, return_sequences=True),
    BatchNormalization(),
    Dropout(dropout),
    Activation(activations.relu),

    Dense(hidden_dim, activation='relu'),
    BatchNormalization(),
    Dropout(dropout),

    Dense(len(tags)+1, activation='softmax') # len(tags)+1 так как еще 0, который показывает где нет тега
])

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          8502272   
                                                                 
 batch_normalization (BatchN  (None, None, 64)         256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 lstm (LSTM)                 (None, None, 64)          33024     
                                                                 
 batch_normalization_1 (Batc  (None, None, 64)         256       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, None, 64)          0

In [38]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [39]:
with tf.device("GPU:0"):
  hh = model.fit(x_train_nn_array, y_train_nn_array, epochs=50, batch_size=512)

Epoch 1/50
136/136 [==============================] - 14s 41ms/step - loss: 1.2960 - accuracy: 0.8095
Epoch 2/50
136/136 [==============================] - 6s 40ms/step - loss: 0.1926 - accuracy: 0.9744
Epoch 3/50
136/136 [==============================] - 6s 41ms/step - loss: 0.0688 - accuracy: 0.9888
Epoch 4/50
136/136 [==============================] - 6s 41ms/step - loss: 0.0355 - accuracy: 0.9936
Epoch 5/50
136/136 [==============================] - 6s 41ms/step - loss: 0.0241 - accuracy: 0.9953
Epoch 6/50
136/136 [==============================] - 6s 41ms/step - loss: 0.0187 - accuracy: 0.9960
Epoch 7/50
136/136 [==============================] - 6s 41ms/step - loss: 0.0156 - accuracy: 0.9965
Epoch 8/50
136/136 [==============================] - 6s 41ms/step - loss: 0.0135 - accuracy: 0.9967
Epoch 9/50
136/136 [==============================] - 6s 41ms/step - loss: 0.0120 - accuracy: 0.9970
Epoch 10/50
136/136 [==============================] - 6s 41ms/step - loss: 0.0108 - accur

Здесь вычисляется accuracy с учётом нулей, нас это не устраивает, поэтому вычислим заново, а так же вычислим для test:

In [48]:
nn_preds_train = model.predict(x_train_nn_array, verbose=1, batch_size=512)
nn_preds_train[0][:2]

array([[4.29459996e-30, 2.03175379e-08, 6.78931356e-10, 5.78906212e-09,
        2.08379702e-10, 1.27422051e-09, 2.18958546e-10, 1.86185767e-09,
        9.99999166e-01, 6.64112108e-08, 1.93452129e-11, 2.76208206e-11,
        7.23405094e-07, 8.41172479e-11, 1.23960953e-09, 1.22484565e-08,
        2.63030664e-09, 2.00094519e-08],
       [0.00000000e+00, 6.61685859e-17, 1.72875139e-19, 2.75030595e-13,
        8.31908056e-14, 4.11649447e-13, 1.51137552e-13, 1.04265235e-10,
        3.90433400e-15, 6.01206064e-12, 1.81535593e-14, 2.14096007e-13,
        5.79462606e-12, 1.00000000e+00, 3.46063907e-11, 5.76627081e-15,
        2.44992806e-12, 2.65939465e-10]], dtype=float32)

In [49]:
acc_train = 0
for i in range(len(x_train_nn_array)):
  len_sent = len(full_train[i])
  acc_train += sum(np.argmax(nn_preds_train[i][:len_sent], axis=1) == y_train_nn_array[i][:len_sent])/len_sent
acc_train/len(x_train_nn_array)

0.9936772111587017

In [50]:
nn_preds_test = model.predict(x_test_nn_array, verbose=1, batch_size=512)
nn_preds_test[0][:2]

array([[9.9787211e-01, 9.9606645e-05, 6.7013963e-05, 3.3477211e-04,
        1.5539853e-04, 9.9225312e-05, 2.4556992e-05, 7.8378971e-06,
        1.4982490e-04, 1.2260582e-04, 2.1979339e-04, 1.9358009e-05,
        1.6564654e-05, 1.0609691e-05, 8.3820749e-05, 4.0918429e-05,
        6.7130697e-04, 4.6177888e-06],
       [0.0000000e+00, 3.7417376e-14, 1.4636751e-16, 1.4374426e-11,
        5.6919526e-12, 2.8832406e-10, 1.0472878e-11, 1.9781545e-09,
        2.4050903e-13, 1.8677841e-10, 6.7347300e-12, 5.1299073e-12,
        4.2676338e-11, 1.0000000e+00, 1.7400141e-09, 9.2168916e-13,
        3.7408084e-11, 1.8618272e-09]], dtype=float32)

In [51]:
acc_test = 0
for i in range(len(x_test_nn_array)):
  len_sent = len(full_test[i])
  acc_test += sum(np.argmax(nn_preds_test[i][:len_sent], axis=1) == y_test_nn_array[i][:len_sent])/len_sent
acc_test/len(x_test_nn_array)

0.8962984384739782

### Вывод:

Лучше всех отработал комбинированный теггер, нейронка тоже неплохо справилась, но при этом для неё нужно подготовить данные, построить модель, обучить и предсказать, это всё оказалось гораааздо дольше, чем раюотать с теггерами из коробки, хотя при этом, комбинированный теггер показал метрику даже лучше.

## Задание 2

### Данные:

In [88]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-09-05 16:21:50--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip.1’

collection5.zip.1   100%[===================>]   1.81M   943KB/s    in 2.0s    

2022-09-05 16:21:53 (943 KB/s) - ‘collection5.zip.1’ saved [1899530/1899530]



In [89]:
!unzip collection5.zip

Archive:  collection5.zip
replace Collection5/001.ann? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [90]:
!pip install -qq corus

In [91]:
from corus import load_ne5

path_coll5 = 'Collection5/'
records = load_ne5(path_coll5)
next(records)

Ne5Markup(
    id='last_66',
    text='\r\nВ департаментах правительства Москвы произошли кадровые перестановки\r\n\r\nВрио мэра Москвы Сергей Собянин произвел кадровые перестановки в столичных департаментах и ведомствах. Как сообщили корреспонденту ИА REGNUM в информационном центре правительства Москвы, заместителем начальника главного архивного управления города Москвы с 1 августа назначен Михаил Горинов, а заместителем руководителя департамента науки, промышленной политики и предпринимательства Григорий Сенчень. С Михаилом Гориновым заключен служебный контракт сроком на один год, а с Григорием Сенченым - на пять лет.\r\n\r\nСергей Золотухин назначен заместителем начальника управления по обеспечению деятельности мировых судей с заключением служебного контракта до 22 августа 2013 года. Ранее Сергей Золотухин был помощником мэра Москвы отдела организации совещаний у мэра.\r\n\r\nС заместителем префекта Центрального административного округа Александром Литошиным продлен срок действия сл

In [92]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [93]:
from razdel import tokenize

In [94]:
words_docs = []
text_data = []
for ix, rec in enumerate(records):
    text_data.append(rec.text)
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [96]:
import pandas as pd

In [97]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [98]:
df_words['tag'].value_counts()

OUT         219110
PER          21184
ORG          13651
LOC           4560
GEOPOLIT      4356
MEDIA         2480
Name: tag, dtype: int64

In [99]:
df_words.head(3)

,word,tag
0,Мэр,OUT
1,Москвы,LOC
2,Сергей,PER


In [100]:
df_words.shape

(265341, 2)

### NLTK

С помощью функции nltk.ne_chunk () мы можем распознавать именованные сущности с помощью классификатора, который добавляет метки категорий, такие как PERSON, ORGANIZATION и GPE.

In [101]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [104]:
from tqdm import tqdm

In [105]:
result_nltk = []
for text in tqdm(text_data):
  result_nltk.append({(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label')})

100%|██████████| 999/999 [01:03<00:00, 15.85it/s]


In [108]:
result_nltk[:2]

[{('Кроме', 'PERSON'),
  ('Леонида Сидорова', 'PERSON'),
  ('Москвы', 'PERSON'),
  ('Москвы Сергей Собянин', 'PERSON'),
  ('Новости', 'PERSON'),
  ('Своим', 'PERSON'),
  ('Сергей Байдаков', 'PERSON'),
  ('Собянин', 'PERSON'),
  ('ЦАО', 'ORGANIZATION')},
 {('AFI Development', 'ORGANIZATION'),
  ('Africa Israel Investment', 'PERSON'),
  ('LSE', 'ORGANIZATION'),
  ('Алмазный', 'PERSON'),
  ('Леваев', 'PERSON'),
  ('Леваева', 'PERSON'),
  ('Москвы', 'PERSON'),
  ('Новости Игорь Катаев МОСКВА', 'PERSON'),
  ('РИА Новости', 'ORGANIZATION'),
  ('Теперь Леваеву', 'PERSON')}]

### Spacy

In [119]:
!pip -qq install spacy
!python -m spacy download en
!python -m spacy download en_core_web_sm

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.4.0/en_core_web_sm-3.4.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [120]:
!pip install -qq -U spacy
!python -m spacy info
import spacy
from spacy import displacy
import en_core_web_sm


============================== Info about spaCy ==============================

spaCy version    3.4.1                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines        en_core_web_sm (3.4.0)        



In [121]:
nlp = en_core_web_sm.load()
result_spacy = []
for text in tqdm(text_data):
  result_spacy.append(nlp(text))

100%|██████████| 999/999 [00:46<00:00, 21.37it/s]


In [126]:
for article in result_spacy[:2]:
  displacy.render(article, jupyter=True, style='ent')

### Deeppavlov

Используем предтренированную модель "ner_ontonotes_bert_mult", которая работает с разными языками, в том числе и русским. Подадим на распознование предложение на русском языке.

In [134]:
 !pip install -qq deeppavlov

Error: Command '['/content/env/bin/python3', '-Im', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-09-05 16:46:46.445 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/squad/squad_bert.json'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-oid9o29c
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-oid9o29c
  fatal: could not read Username for 'https://github.com': No such device or address
ERROR: Command errored out with exit status 128: git clone -q https://github.com/deepmipt/ber

In [129]:
import deeppavlov
from deeppavlov import configs, build_model

In [136]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 26.2 MB/s 
     |████████████████████████████████| 6.6 MB 58.2 MB/s 
     |████████████████████████████████| 120 kB 60.9 MB/s 


In [ ]:
deeppavlov_ner = build_model(configs.ner.ner_ontonotes_bert_torch, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])

2022-09-05 16:54:57.741 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_torch.tar.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_torch.tar.gz download because of matching hashes


2022-09-05 16:55:13.258 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_torch/bert-base-cased/tag.dict]
INFO:deeppavlov.core.data.simple_vocab:[loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_torch/bert-base-cased/tag.dict]


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas